<a href="https://colab.research.google.com/github/abhisek-ai/py/blob/master/RSS_news_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install feedparser
import feedparser
from sqlalchemy import create_engine, Column, String, DateTime, Integer, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [5]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
Base = declarative_base()

<ipython-input-6-9bae6e3862ba>:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [7]:
class NewsArticle(Base):
    __tablename__ = 'news_articles'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    content = Column(String)
    pub_date = Column(DateTime)
    source_url = Column(String)
    category = Column(String)

In [8]:
# Connect to SQLite in-memory database (for demonstration)
engine = create_engine('sqlite:///:memory:')
Base.metadata.create_all(engine)

In [9]:
Session = sessionmaker(bind=engine)

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

In [12]:
def process_text(text):
    words = word_tokenize(text)
    filtered_words = [ps.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [13]:
def classify_category(text):
    # Implement your own classification logic here using NLTK or spaCy
    # This is a placeholder
    return 'Positive/Uplifting'

In [14]:
def process_article(session, article_data):
    try:
        article = NewsArticle(**article_data)
        session.add(article)
        session.commit()

        # Perform NLP classification
        processed_content = process_text(article.content)
        category = classify_category(processed_content)

        article.category = category
        session.commit()
    except Exception as e:
        # Handle errors and log appropriately
        print(f"Error processing article: {e}")

In [22]:
import feedparser

rss_feed = 'http://rss.cnn.com/rss/cnn_topstories.rss'
feed = feedparser.parse(rss_feed)

for entry in feed.entries:
    # Check if 'published' exists in the entry
    if 'published' in entry:
        pub_date = entry.published
    else:
        pub_date = None

    # Check if 'summary' exists in the entry
    if 'summary' in entry:
        summary = entry.summary
    else:
        summary = None

    article_data = {
        'title': entry.title,
        'content': summary,
        'pub_date': pub_date,
        'source_url': entry.link,
    }


In [23]:
# Query the database for results
result = session.query(NewsArticle).all()
for article in result:
    print(article.title, article.category)

session.close()